In [ ]:
import pickle
import numpy as np
from matplotlib import pyplot as plt

from lfd import compute_estimates
from plotting import plot_est_and_gt_ellipses_on_images, plot_3D_scene


: 

In [2]:
dataset = 'tiger'

Load the input data.
Data association is implicitly defined in the data structures: each column of "visibility" corresponds to one object.
The information in "bbs" is structured in a similar way, with four columns for each object.

In [3]:
bbs = np.load('Data/{:s}/InputData/bounding_boxes.npy'.format(dataset))
K = np.load('Data/{:s}/InputData/intrinsics.npy'.format(dataset)) 
Ms_t = np.load('Data/{:s}/InputData/camera_poses.npy'.format(dataset))
visibility = np.load('Data/{:s}/InputData/visibility.npy'.format(dataset))

Compute the number of frames and the number of objects for the current dataset from the size of the visibility matrix.


In [4]:
n_frames = visibility.shape[0]
n_objects = visibility.shape[1]

In [5]:
[inputCs, estCs, estQs] = compute_estimates(bbs, K, Ms_t, visibility)

In [6]:
Qs = estQs[0]

In [9]:
def dual_quadric_to_ellipsoid_parameters(Q):
    """Computes centre, axes length and orientation of one ellipsoid.

   A [4x4] matrix can represent general quadrics. In spite of preconditioning, the estimated quadrics can still,
   in rare cases, represent something other than ellipsoids. This is corrected by forcing the lengths of the
   axes to be positive.

    :param Q: Ellipsoid/Quadric in dual form [4x4].

    :returns:
      - centre - Ellipsoid centre in Cartesian coordinates [3x1].
      - axes - Length of ellipsoid axes [3x1].
      - R - Orientation of the ellipsoid [3x3].
    """

    # Scale the ellipsoid to put it in the usual form, with Q[3,3] = -1.
    Q = Q / (-Q[3, 3])

    # Compute ellipsoid centred on origin.
    centre = -Q[:3, 3]
    T = np.vstack((np.array((1, 0, 0, -centre[0])), np.array((0, 1, 0, -centre[1])),
                   np.array((0, 0, 1, -centre[2])), np.array((0, 0, 0, 1))))
    Qcent = T.dot(Q).dot(T.transpose())

    # Compute axes and orientation.
    [D, V] = np.linalg.eig(Qcent[:3, :3])
    sort_ev = np.argsort(D)
    V = np.vstack((V[:, sort_ev[0]], V[:, sort_ev[1]], V[:, sort_ev[2]])).transpose()
    D.sort()
    if sum(D < 0) > 0:
        # Take the absolute value of eigenvalues (they can be negative because of numerical issues).
        for index in range(D.shape[0]):
            if D[index] < 0:
                D[index] *= -1

    a = np.sqrt(D[0])
    b = np.sqrt(D[1])
    c = np.sqrt(D[2])
    
    axes = np.array([a, b, c])
    R = V
    return centre, axes, R

SyntaxError: invalid syntax (1777738030.py, line 1)